In [ ]:
%load_ext autoreload
%autoreload 2

import re
import os
import json
import time
import sqlite3
import datetime
import operator
import requests

import openreview

import numpy as np
from tqdm.notebook import tqdm, trange

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# from src import DataBase

In [ ]:
# ids from api
client = openreview.Client(baseurl='https://api.openreview.net')
notes = openreview.tools.iterget_notes(client, invitation='ICLR.cc/2023/Conference/-/Blind_Submission')

ids = []
for note in notes:
    ids.append(note.id)

In [ ]:
def get_item_url(panel, url_list, page_id):
    # find all items in current page
    item_list = panel.find_elements(By.CLASS_NAME, "note")
    item_list_len = len(item_list)
    for i in trange(item_list_len, desc=f"->{page_id}"):
        # the fist  is the paper title and url
        item = item_list[i].find_element(By.TAG_NAME, 'a')
        url_list.append(item.get_attribute('href').strip()) 
        
        
def get_url_list_for_tab(panel):
    # get totoal page number
    _right_arrow = panel.find_elements(By.CLASS_NAME, 'right-arrow')[-1]
    total_pages = int(_right_arrow.get_attribute('data-page-number'))
    
    # loop all pages
    url_list = []
    for page_id in trange(total_pages):
        get_item_url(panel, url_list, page_id)
        
        if page_id == total_pages - 1:
            continue
        
        # the first item
        flag = panel.find_element(By.CLASS_NAME, "note").text
        
        time.sleep(1.5)
        
        # click to jump to next page
        next_page_btn = panel.find_element(By.CSS_SELECTOR, "li[class='  right-arrow']")
        next_page_btn.find_element(By.TAG_NAME, 'a').click()
        
        # jump to next page
        jumped = False
        num_try = 0
        while not jumped:
            time.sleep(1)
            _flag = panel.find_element(By.CLASS_NAME, "note").text
            if _flag != flag:
                jumped = True
            num_try += 1
            if num_try > 1000:
                break
    
    return url_list
    

In [ ]:
s = Service('/opt/homebrew/bin/chromedriver')
op = Options()
op.add_argument('headless')
driver = webdriver.Chrome(service=s)
driver.get(f'https://openreview.net/group?id=ICLR.cc/2023/Conference#withdrawn-submissions')
time.sleep(2)
tabpanel = driver.find_element(By.ID, f'withdrawn-submissions')
url_list = get_url_list_for_tab(tabpanel)
# quit
driver.quit()

In [ ]:
# all submission ids
sub_ids = []
sub_ids.extend(ids)
# withdrawn
for x in url_list:
    sub_ids.append(x.split('id=')[1])
print(len(sub_ids))

In [35]:
# filter
filter_ids = []
for i in trange(len(sub_ids)):
    item_id = sub_ids[i]
    rqst_url = f"https://api.openreview.net/notes?forum={item_id}"
    rqst_data = requests.get(rqst_url)
    rqst_data = json.loads(rqst_data.text)

    # parse each note
    meta_note = [d for d in rqst_data['notes'] if 'Paper' not in d['invitation']]
    # filter reviewer comments
    comment_notes = [d for d in rqst_data['notes'] \
                     if 'Official_Review' in d['invitation'] and 'recommendation' in d['content'].keys()]
    if len(comment_notes) > 0:
        filter_ids.append(item_id)

  0%|          | 0/4941 [00:00<?, ?it/s]

In [37]:
# save to file
save_path = f"../assets/filter_ids_" \
            f"{datetime.datetime.now():%Y-%m-%d-%H:%M-UTC}.txt"
with open(save_path, 'w') as f:
    f.write('\n'.join(filter_ids))  

test fetch data

In [38]:
with open('../assets/filter_ids_2022-11-05-19:32-UTC.txt', 'r') as f:
    data = f.readlines()

In [81]:
url = f"https://openreview.net/forum?id={data[9].strip()}"
url = "https://openreview.net/forum?id=diOVflNRZnG"
print(url)

https://openreview.net/forum?id=diOVflNRZnG


In [82]:
item_id = url.split('id=')[-1]
rqst_url = f"https://api.openreview.net/notes?forum={item_id}"
rqst_data = requests.get(rqst_url)
rqst_data = json.loads(rqst_data.text)

In [83]:
# parse each note
withdraw = 0
# filter meta note
meta_note = [d for d in rqst_data['notes'] if 'Paper' not in d['invitation']]
# check withdrawn
withdraw = 1 if 'Withdrawn_Submission' in meta_note[0]['invitation'] else 0
# decision
# if withdraw == 0:
#     decision_note = [d for d in rqst_data['notes'] if 'Decision' in d['invitation']]
#     decision = decision_note[0]['content']['decision']
# else:
#     decision = ''
# filter reviewer comments
comment_notes = [d for d in rqst_data['notes'] \
                 if 'Official_Review' in d['invitation'] and 'recommendation' in d['content'].keys()]
comment_notes = sorted(comment_notes, key=lambda d: d['number'])[::-1]
ratings = [int(note['content']['recommendation'].split(':')[0]) for note in comment_notes]

In [84]:
title = meta_note[0]['content']['title']
print(title)
authors = meta_note[0]['content']['authors']
print(authors)
keywords = meta_note[0]['content']['keywords']
print(keywords)

Curvature Informed Furthest Point Sampling
['Shubham Bhardwaj', 'Balakrisha Reddy Pailla']
['dowsampling', 'point cloud', 'curvature informed', 'shape completion', 'segmentation', 'furthest point sampling']
